In [419]:
def removeUnnecessaryParentheses(tree_string, prevLength, prevLocation):
    '''
    Given a string, remove all unncessary parentheses.
    
    For example, "(((((((InputLayer)))))))" becomes "(InputLayer)"
  
    Parameters:
    tree_string (str): a string
    prevLength (int): the length of thetree_string in the previous recursive call
    prevLocation (int): the index of the previous open parenthesis in the previous recursive call
  
    Returns:
    str: an updated string containing only the necessary parentheses
    '''
    
    # find rightmost "(" before the first ")"
    prevOpenParenIndex = prevLocation
    i = prevLocation
    while (i < len(tree_string)):
        # if ")" is found BUT there is a "(" in the remainder of the string AND there is no previous "("...
            # we should continue searching for a "("
        # otherwise, stop looking
        if (tree_string[i] == ")" and "(" in tree_string[i + 1: len(tree_string)] and tree_string[prevOpenParenIndex] != "("):
            i = i + 1
        elif (tree_string[i] == ")"):
            break
        
        # keep adf parentheses
        # ex. "adf_3(InputLayer)" should not become "adf_3InputLayer"
        if (tree_string[i] == "("):
            if (tree_string[i - 5: i - 2] != "adf"):
                prevOpenParenIndex = i
        i = i + 1
        
    # find leftmost rightmost ")" that is a pair with the above "("
    j = prevOpenParenIndex
    while (j < len(tree_string) and tree_string[j] != " "):
        if (tree_string[j] == ")"):
            tree_string = tree_string[:prevOpenParenIndex] + tree_string[prevOpenParenIndex + 1:]
            
            # account for shift of string
            j = j - 1
            
            tree_string = tree_string[:j] + tree_string[j + 1:]
            break
            
        j = j + 1
        
    if (len(tree_string) == prevLength):
        return tree_string
    
    prevLength = len(tree_string)
    return removeUnnecessaryParentheses(tree_string, prevLength, prevOpenParenIndex - 1)

In [420]:
def create_adfs_dict(adfs):
    '''
    Given a list of adfs, return a dict of { adf_* : adf_string }
    
    For example, if 'adf_3:Conv2DLayer(InputLayer)', the dict will contain { adf_3 : Conv2DLayer(InputLayer) }
  
    Parameters:
    adfs (list): a list of adfs
  
    Returns:
    dict: contains adf_* for keys, adf_string for values
    '''
    adf_dict = dict()
    
    for i in adfs:
        if ("adf_" not in i):
            raise ValueError("adf at index %d does not contain string of 'adf_*'" % (i))
            return res
        
        key_value = i.split(":")
        adf_dict[key_value[0]] = key_value[1]
        
    return adf_dict

In [421]:
def findStringWithinMatchingParenthesis(base_string, adf_number_index):
    '''
    Given a base string, the adf_*, and starting index of adf_*,
    find the starting and ending index of the string within the adf_*(...)
    
    For example, if base_string is 'adf_3(InputLayer)', returns (6, 16)
                                          ^         ^
    
    Parameters:
    base_string (str): ex. NNLearner(ARG0, adf_8(FlatternLayer4d(adf_3(InputLayer))))
                                           ^
    adf_number_index: ex. 16 (for adf_8)
  
    Returns:
    (str, str): start index of string within adf_* (inclusive), end index of string within adf_* (exclusive)
    '''
    numAdditionalOpenParen = 0

    prevOpenParenIndex = -1
    i = adf_number_index
    while (i < len(base_string)):
        if (numAdditionalOpenParen == 0 and base_string[i] == ")"):
            break
        elif (base_string[i] == ")"):
            numAdditionalOpenParen = numAdditionalOpenParen - 1
        
        if (prevOpenParenIndex == -1 and base_string[i] == "("):
            prevOpenParenIndex = i
        elif (base_string[i] == "("):
            numAdditionalOpenParen = numAdditionalOpenParen + 1
        i = i + 1
    
#     print("findStringWithin: ", base_string[prevOpenParenIndex + 1:i])
    return (prevOpenParenIndex + 1, i)

In [422]:
def insert_adfs(base_string, adf_dict):
    '''
    Given a base string, replaces all instances of adf_* with its proper string located in adf_dict.
    
    Parameters:
    base_string (str): ex. NNLearner(ARG0 , adf_8(DenseLayerUnit256, adf_3(InputLayer)))
    adf_dict (dict): ex. adf_dict: {
                             adf_8: SomeLayer(defaultActivation, InputLayer),
                             adf_3: SomeLayer(InputLayer)
                         }
  
    Returns:
    str: base_string with all instances of adf_* replaced with its proper string
                ex. NNLearner(ARG0 , SomeLayer(defaultActivation, DenseLayerUnit256, SomeLayer(InputLayer)))
    '''
    i = 0
    while(i < len(base_string) - 4 and "adf_" in base_string):
        # find "adf_*"
        # find all values within the adf_*( ... ) -> done with findStringWithinMatchingParenthesis(base_string, i)
        # replace "InputLayer" in adf_dict[adf_*(...)] with values within adf_* from base_string
        
        if (base_string[i: i + 4] == "adf_"):
            adf_number = base_string[i: i + 5]
            
            # find matching open and close parenthesis
            start_replace, end_replace = findStringWithinMatchingParenthesis(base_string, i)
            
            replacement = adf_dict[adf_number].replace("InputLayer", base_string[start_replace:end_replace])
            base_string = base_string[:start_replace - 6] + replacement + base_string[end_replace + 1:]
            
        i = i + 1
    return base_string

In [423]:
def parse_adf_tree_string(string):
    # prepare string
    tree_string_arr = tree_string.replace(" ", "").replace("\n", " ").split(" ")

    for i in range(len(tree_string_arr)):
        tree_string_arr[i] = tree_string_arr[i].replace(",", " , ")

    print("---------------------------------------")
    print("preparing string:")
    print("---------------------------------------")
    for i in tree_string_arr:
        print(i)
        print("\n")
    
    # remove unncessary parentheses
    print("---------------------------------------")
    print("removing unnecessary parentheses:")
    print("---------------------------------------")
    for i in range(len(tree_string_arr)):
        tree_string_arr[i] = removeUnnecessaryParentheses(tree_string_arr[i], len(tree_string_arr), -1)
        print(tree_string_arr[i])
        print("\n")
    
    # generate final string
    adf_dict = create_adfs_dict(tree_string_arr[1:])
    final_string = insert_adfs(tree_string_arr[0], adf_dict).replace(" , ", ", ")
    print("---------------------------------------")
    print("final string:")
    print("---------------------------------------")
    print(final_string)
#     return final_string

In [424]:
string = "NNLearner((((ARG0))), FlattenLayer4d(adf_8(Conv2DLayer(Conv2DFilterUnit16, ((reluActivation)), (((((((((Conv2DKernelSize3))))))))), adf_3(InputLayer)))), (((((DenseLayerUnit256))))), (((((NadamOptimizer))))))\nadf_3: MaxPoolingLayer2D(Conv2DKernelSize5, Conv2DLayer(Conv2DFilterUnit48, defaultActivation, Conv2DKernelSize3, InputLayer))\nadf_8: DenseLayer4dim((DenseLayerUnit256), (defaultActivation), Conv2DLayer(Conv2DFilterUnit16, sigmoidActivation, ((Conv2DKernelSize5)), InputLayer))"
parse_adf_tree_string(string)
# print(parse_adf_tree_string(string))

---------------------------------------
preparing string:
---------------------------------------
NNLearner((((ARG0))) , FlattenLayer4d(adf_8(Conv2DLayer(Conv2DFilterUnit16 , ((reluActivation)) , (((((((((Conv2DKernelSize3))))))))) , adf_3(InputLayer)))) , (((((DenseLayerUnit256))))) , (((((NadamOptimizer))))))


adf_3:MaxPoolingLayer2D(Conv2DKernelSize5 , Conv2DLayer(Conv2DFilterUnit48 , defaultActivation , Conv2DKernelSize3 , InputLayer))


adf_8:DenseLayer4dim((DenseLayerUnit256) , (defaultActivation) , Conv2DLayer(Conv2DFilterUnit16 , sigmoidActivation , ((Conv2DKernelSize5)) , InputLayer))


---------------------------------------
removing unnecessary parentheses:
---------------------------------------
NNLearner(ARG0 , FlattenLayer4d(adf_8(Conv2DLayer(Conv2DFilterUnit16 , reluActivation , Conv2DKernelSize3 , adf_3(InputLayer)))) , DenseLayerUnit256 , NadamOptimizer)


adf_3:MaxPoolingLayer2D(Conv2DKernelSize5 , Conv2DLayer(Conv2DFilterUnit48 , defaultActivation , Conv2DKernelSize